<a href="https://colab.research.google.com/github/ArpitaChatterjee/Decision-Tree-Classifier/blob/main/Decision_Tree_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#set my sample dataset
train_data=[
    ['green', 3, 'mango'],
    ['yellow', 3, 'mango'],
    ['red', 1, 'grape'],
    ['red', 1, 'grape'],
    ['yellow', 3, 'lemon'],
]
#col labels to print tree
header=['color', 'diameter', 'label']


In [8]:

def unique_val(rows, col):
    #find unique values for col in dataset
    return set([row[col] for row in rows])
##
#demo:
#unique_val(train_data, 0)..
def class_counts(rows):
    """counts no of eaxch type in dataset"""
    counts={}#dictionary of labels->count
    for row in rows:
        #in dataset labels is always the last col
        label=row[-1]
        if label not in counts:
            counts[label]=0
        counts[label]+=1
    return counts

#demo:
#class_count(train_data)

def is_numeric(value):
    """test if value is numeric or not"""
    return isinstance(value, int) or isinstance(value, float)
#demo:
#is_numeric(7)
#isnumeric('red')

In [12]:
class question:
    """a q. is usd to partition a dataset"""
    
    def __init__(self, column ,value):
        self.column=column
        self.value=value
    
    def match(self, example):
        #compare featur val in eq with those in q.
        val= example[self.column]
        if is_numeric(val):
            return val>= self.value
        else:
            return val== self.value
        
    def __repr__(self):
        #mthd to print the q. in readable mthd
        condition="=="
        if is_numeric(self.value):
            condition=">="
        return "is %s %s %s?" %(
        header[self.column], condition, str(self.value))


def partition(rows, question):
    '''dataset for each row in the dataset to check if it matches the q. if so 
    add it to the true rows otherwise false rows'''
    
    true_rows, false_rows= [] , []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

##partition traindatast if rows r red or not
#true_rows, false_rows = partition(training_data, question(0, 'red'))
#true_row==contain all "red" rows
#false_rows == cpntain everything else

def gini(rows):
    '''calculate gini impurity for a list of rows'''
    counts=class_counts(rows)
    impurity =1
    for lbl in counts:
        prob_of_lbl = counts[lbl]/float(len(rows))
        impurity -=prob_of_lbl**2
    return impurity



In [13]:

def info_gain(left, right, curr_uncertainity):
    '''Information Gain== the uncertainity of the starting node- wt. impurity of 2 child node'''
    
    p= float(len(left))/ (len(left)+len(right))
    return curr_uncertainity-p*gini(left)-(1-p)*gini(right)

##calculate uncertainity of our training dataset
#curr_uncertainity = gini(training_data)

#ig when partition is on 'green'
#true_rows, false_rows = partition(training_data, question(0, 'green'))
#ingo_gain(true_rows, false_rows, curr_uncertainity)

#ig when partition is on 'red'
##true_rows, false_rows = partition(training_data, question(0, 'red'))
#ingo_gain(true_rows, false_rows, curr_uncertainity)


def find_best_split(rows):
    '''find the best q. to ask by iterationg over every feature / value and calcualting the ig'''
    best_gain= 0 #to keep track of best ig
    best_question = None #keep train of the feature value that produced it
    curr_uncertainity = gini(rows)
    n_features = len(rows[0])-1 #no of col
    for col in range(n_features):
        values = set([row[col] for row in rows]) #unique values in col
        for val in values:#foreach val
            ques = question(col, val) 
            
            #try spliting the dataset
            true_rows, false_rows = partition(rows, question)
            
            #skip the split if it doesnt divide the dataset
            
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
                
            #ig from this split
            gain= info_gain(true_rows, false_rows, curr_uncertainity)
            
            #use '>' in stead of '>=' in the below code
            if gain >= best_gain:
                best_gain, best_question= gain, ques
    return best_gain, best_question

#demo:
#to find the best q. to ask 1st for our toydataset
#best_gain, best_question = first_best_split(training_data)
#fyi: is color == 'red' is just as good

class leaf:
    '''a leaf node classifies data'''
    def __init__(self, rows):
        self.prediction = class_counts(rows)
        
        
class decision_node:
    '''a decision node asks a q. --which holds reference to the q., and to the 2 child nodes'''
    
    def __init__(self, ques, true_branch, false_branch):
        self.ques= ques
        self.true_branch = true_branch
        self.false_branch = false_branch
        

In [14]:

def build_tree(rows):
    '''build the tree'''
    
    #ty partition the datset on ech unique attribute,
    #calc the ig, n return q. that produces the highest gain
    gain, ques = find_best_split(rows)
    
    #base case: no further info gain, since we can ask no further q. well return a leaf
    if gain==0:
        return leaf(rows)
    
    #if we reach here we have found a useful feature/ value to partition on
    true_rows, false_rows = partition(rows, ques)
    
    #recursively build the true branch
    true_branch = build_tree(true_rows)
    
    #return a q. node
    #this records the best feature/ value to ask at this pt. 
    #as well branches to follow depending on the ans.
    return decision_node(ques, true_branch, false_branch)


In [ ]:
def print_tree(node, spacing=""):
    '''best elegant tree printing fn'''

    #base case: we've reached a leaf
    if isinstance(node, leaf):
        print(spacing + "predict ", node.predictions)
        return
    
    #print the q.
    print(spacing + str(node.ques))
    
    #call this fn recursively on the true branch
    print(spacing + '-->True:')
    print_tree(node.true_branch, spacing+' ')
    
    ##call this fn recursively on the false branch
    print(spacing + '-->True:')
    print_tree(node.false_branch, spacing+' ')
    
def classify(row, node):
    
    #basecase: we've reached a leaf
    if isinstance(node, leaf):
        return node.predictions
    
    #decide whether to follow the true-branch or false-branch
    #compare feature/value stored in the node to the eg we're considering
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)
    
#demo:printing that a bit nicer
#print_leaf(classify(training_data[0], my_train))

if __name__=='__main__':
    
    my_tree = build_tree(train_data)
    
    print_tree(my_tree)
    
    #evaluate
    testing_data=[
        ['green', 3, 'mango'],
    ['yellow', 3, 'mango'],
    ['red', 1, 'grape'],
    ['red', 1, 'grape'],
    ['yellow', 3, 'lemon'],]
    
    for row in testing_data:
        print("Actual: %s. Predicted: %s" %
             (row[-1], print_leaf(classify(row, my_tree))))
        
#for next step add support for missing(or unseen) attribute
#prune the tree to prevent overfitting
#add supportfor regression
